In [114]:
from PIL import Image
from transformers import CLIPProcessor, CLIPTextModel, CLIPTokenizer, AutoTokenizer, CLIPTextModelWithProjection, CLIPVisionModelWithProjection, CLIPModel
import torch
import os
import numpy as np


In [93]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPTextModelWithProjection.from_pretrained("vinid/plip")
processor = CLIPProcessor.from_pretrained("vinid/plip")
tokenizer = AutoTokenizer.from_pretrained("vinid/plip")


/Users/jacksalici/mambaforge/envs/deep/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [129]:
image_embedding_path = '/Users/jacksalici/Documents/University/2024-1_AIxBioInformatics/Project/Repo/embeddings/TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a239-3a24caca9c28/patch_1_embedding.pt' 
image_embeds = torch.load(image_embedding_path).detach()



In [130]:
image_embeds=image_embeds.squeeze(1)

In [115]:
# RUN TO TEST A SINGLE IMAGE INSTEAD OF AN EMBEDDINGS
from PIL import Image

image_path = '/Users/jacksalici/Documents/University/2024-1_AIxBioInformatics/Project/Repo/PLIP/images/image1.jpg'
image = Image.open(image_path)

inputs = processor(images= image, return_tensors="pt", padding=True)

image_model = CLIPVisionModelWithProjection.from_pretrained("vinid/plip")

outputs = image_model(**inputs)
image_embeds = outputs.image_embeds.detach()

/Users/jacksalici/mambaforge/envs/deep/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [133]:
# RUN TO TEST THE ENTIRE MODEL
from PIL import Image

image_path = '/Users/jacksalici/Documents/University/2024-1_AIxBioInformatics/Project/Repo/PLIP/images/image1.jpg'
image = Image.open(image_path)

inputs = processor(images= image, text=['an image of cancer ovarian tissue', 'an image of healthy ovarian tissue'], return_tensors="pt", padding=True)

image_model = CLIPModel.from_pretrained("vinid/plip")

outputs = image_model(**inputs)
probs = outputs.logits_per_image.softmax(dim=1)
probs

ValueError: You have to specify pixel_values

In [127]:
inputs = processor(text=['an image of cancer ovarian tissue', 'an image of healthy ovarian tissue'], return_tensors="pt", padding=True)
outputs =model(**inputs)

In [131]:
text_embeds = outputs.text_embeds.detach()

In [132]:
# normalized features
image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)

logit_scale = torch.nn.Parameter(torch.tensor(2.6592))

# cosine similarity as logits
logit_scale = logit_scale.exp()
logits_per_text = torch.matmul(text_embeds, image_embeds.t()) * logit_scale
logits_per_image = logits_per_text.t()

probs = logits_per_image.softmax(dim=1)
probs


tensor([[0.4987, 0.5013]], grad_fn=<SoftmaxBackward0>)